In [1]:
import numpy as np
from Molecule import Molecule

In [2]:
mol = Molecule('geom.dat')
mol.print_geom()

Number of Atoms : 7
Coordinate File: 
6    0.000000    0.000000    0.000000
6    0.000000    0.000000    2.845112
8    1.899116    0.000000    4.139063
1   -1.894048    0.000000    3.747689
1    1.942501    0.000000   -0.701146
1   -1.007295   -1.669972   -0.705917
1   -1.007295    1.669972   -0.705917


# Calculate Bond Length


In [3]:
for i in range(len(mol.coords)):
    for j in range(0, i):
        print(f'Atom {i}    Atom {j} {(mol.bond(i, j)):10.5f}')

Atom 1    Atom 0    2.84511
Atom 2    Atom 0    4.55395
Atom 2    Atom 1    2.29803
Atom 3    Atom 0    4.19912
Atom 3    Atom 1    2.09811
Atom 3    Atom 2    3.81330
Atom 4    Atom 0    2.06517
Atom 4    Atom 1    4.04342
Atom 4    Atom 2    4.84040
Atom 4    Atom 3    5.87463
Atom 5    Atom 0    2.07407
Atom 5    Atom 1    4.05133
Atom 5    Atom 2    5.89151
Atom 5    Atom 3    4.83836
Atom 5    Atom 4    3.38971
Atom 6    Atom 0    2.07407
Atom 6    Atom 1    4.05133
Atom 6    Atom 2    5.89151
Atom 6    Atom 3    4.83836
Atom 6    Atom 4    3.38971
Atom 6    Atom 5    3.33994


# Bond Angles
The angle, $\theta_{ijk}$, between atoms i-j-k, where j is the central atom is given by:

$cos\theta_{ijk} = e_{ji} e_{jk}$

where  $e_{ij}$ are unit vectors between the atoms, e.g.,

$e_{ij}^x = -(x_i - x_j)/R_{ij}, e_{ij}^y = -(y_i - y_j)/R_{ij}, e_{ij}^z = -(z_i - z_j)/R_{ij}$

In [4]:
for i in range(len(mol.coords)):
    for j in range(0, i):
        for k in range(0, j):
            if (mol.bond(i, j) < 4.0 and mol.bond(j, k) < 4.0):
                print(f'{i}- {j}, {k}, {mol.bond_angle(i, j, k):6f}') 


2- 1, 0, 124.268308
3- 1, 0, 115.479341
3- 2, 1, 28.377448
5- 4, 0, 35.109529
6- 4, 0, 35.109529
6- 5, 0, 36.373677
6- 5, 4, 60.484476


# Dihderal Angle 

In [5]:
for i in range(mol.natom):
    for j in range(i):
        for k in range(j):
            for l in range(k):
                if mol.bond(i, j) < 4.0 and mol.bond(j, k) < 4.0 and mol.bond(k, l) < 4.0:
                    print(f"{i:2d}-{j:2d}-{k:2d}-{l:2d} {mol.dihedral_angle(i, j, k, l):10.6f}")


 3- 2- 1- 0 180.000000
 6- 5- 4- 0  36.366799


# Out of Plane Angle

### Calculate all possible out-of-plane angles. For example, the angle $θ_{ijkl}$ for atom i out of the plane containing atoms j-k-l (with k as the central atom, connected to i)

In [11]:
def unit_vector(i, j):
    # i, j are numpy array
    vec = j - i 
    return vec/np.linalg.norm(vec)




def oop(i, j, k ,l):
    # i, j, k, l are numpy array of four atoms 

    res = np.cross(unit_vector(k, j), unit_vector(k, l)).dot(unit_vector(k, i))
    res = res/(np.sin(np.degrees(mol.bond_angle(j, k, l))))
    # assert(np.abs(res) < 1 + 1e-7)
    # res = np.sign(res) if np.abs(res) > 1 else res
    res = np.clip(res, -1.0, 1.0)

    res = np.arcsin(res) 

    return (np.degrees(res))

i = np.array([0.000000000000,     0.000000000000,     0.000000000000])
j = np.array([-1.007295466862,     1.669971842687,    -0.705916966833])
k = np.array([1.942500819960,     0.000000000000,    -0.701145981971])
l = np.array([-1.007295466862,    -1.669971842687,    -0.705916966833])

print(oop(i, j, k ,l))

17.85468937184374


# Center 0f mass Translation 

In [6]:
masses = {6:12.0000000, 8:15.994914619, 1:1.00782503223}

# Calculate Molecular Mass
M = 0
for i in range(len(mol.atoms)):
    M += masses[int(mol.atoms[i])]

print(f'Molecular Mass of the Molecule is {M:6f}')


xcm = 0
ycm = 0
zcm = 0

for i, j in zip(mol.atoms, mol.coords):
    # print(i, masses[int(i)]) 
    xcm += masses[int(i)]*j[0]
    ycm += masses[int(i)]*j[1]
    zcm += masses[int(i)]*j[2]

xcm = xcm/M
ycm = ycm/M
zcm = zcm/M 



print(f'Molecular Center of mass:   {xcm:6f}    {ycm:6f}    {zcm:6f}')




Molecular Mass of the Molecule is 44.026215
Molecular Center of mass:   0.644949    0.000000    2.316638


In [7]:
mol.translate(-xcm, -ycm, -zcm)
print(mol.coords)

[[-0.64494925  0.         -2.3166379 ]
 [-0.64494925  0.          0.52847423]
 [ 1.25416671  0.          1.82242463]
 [-2.53899756  0.          1.43105077]
 [ 1.29755157  0.         -3.01778388]
 [-1.65224472 -1.66997184 -3.02255487]
 [-1.65224472  1.66997184 -3.02255487]]


# Principal Moments of Inertia 

In [12]:
I = np.zeros(shape=(3, 3))
for i, j in zip(mol.atoms, mol.coords):
    I[0][0] += (masses[int(i)] *  (j[1]*j[1] + j[2]*j[2]))
    I[1][1] += (masses[int(i)] *  (j[0]*j[0] + j[2]*j[2]))
    I[2][2] += (masses[int(i)] *  (j[0]*j[0] + j[1]*j[1]))
    I[0][1] -= masses[int(i)] * j[0]*j[1] 
    I[0][2] -= masses[int(i)] * j[0]*j[2] 
    I[1][2] -= masses[int(i)] * j[1]*j[2] 

I[1][0] = I[0][1]
I[2][0] = I[0][2]
I[2][1] = I[1][2]

    # Off Diagonal terms 

print('Moment of Inertia tensor amu bohr^2')
print(I)

eigenvalue, eigenvectors = np.linalg.eigh(I)

print('Principle moments of inertia amu* bohr^2')
print(eigenvalue)
print('Principle moments of inertia amu* AA^2')
conv =  0.529177249 * 0.529177249
print(eigenvalue*conv)

Moment of Inertia tensor amu bohr^2
[[156.15409142   0.         -52.85558333]
 [  0.         199.37112652   0.        ]
 [-52.85558333   0.          54.4595489 ]]
Principle moments of inertia amu* bohr^2
[ 31.96407898 178.64956134 199.37112652]
Principle moments of inertia amu* AA^2
[ 8.95085504 50.02697956 55.82960964]


# Classify the Rotors 

In [13]:
if len(mol.atoms) == 2:
    print('Molecule is diatomic')
elif (eigenvalue[0] < 1e-4):
    print('Molecule is linear')
elif (abs(eigenvalue[0] - eigenvalue[1]) < 1e-4) and (abs(eigenvalue[1] - eigenvalue[2]) < 1e-4):
    print('Molecule is spherical top')
elif (abs(eigenvalue[0] - eigenvalue[1]) < 1e-4) and (abs(eigenvalue[1] - eigenvalue[2]) > 1e-4):
    print('Molecule is oblate symmetric top')
elif (abs(eigenvalue[0] - eigenvalue[1]) > 1e-4) and (abs(eigenvalue[1] - eigenvalue[2]) < 1e-4):
    print('Molecule is prolate symmetric top')
else:
    print('Molecule is asymmetric top')

# Compute rotational constants 

factor = 6.6260755E-34/(8.0*np.pi*np.pi)
factor /= 1.6605402E-27 * 0.529177249E-10 * 0.529177249E-10
factor /= 2.99792458E10

print('Rotational constant in cm-1')
print(f'A = {factor/eigenvalue[0]:.4f} B = {factor/eigenvalue[1]:.4f} C = {factor/eigenvalue[2]:.4f} ')


Molecule is asymmetric top
Rotational constant in cm-1
A = 1.8834 B = 0.3370 C = 0.3019 
